In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [2]:
"""
df = pd.read_excel("./custom-data/population_okinawa.xlsx",
                   skiprows=10, usecols="B,D",
                   )

df.columns = ["place", "population"]
df["population"] = df["population"].str.replace(",", "").astype(int)
df
"""

population = pd.read_excel("./custom-data/population//2018-a.xls", header=2).dropna(how="any", axis=0)
population.columns = population.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
population["市区町村名"] = population["市区町村名"].apply(lambda x: x.replace("\u3000", ""))
population

,市 区 町 村,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,人口集中-地区人口,出生数,死亡数,...,世帯数,一般世帯数,核家族-世帯数,単独世帯数,65歳以上の世帯員の-いる核家族世帯数,高齢夫婦-世帯数,高齢単身-世帯数,婚姻件数,離婚件数,市区-町村-ｺｰﾄﾞ
4,1.0,北海道,5381733,608296,3190804,1558387,21676,4047281,35125,61906,...,2444810,2438206,1363818,909106,558618,327509,319408,24636,10476,1.0
5,1100.0,札幌市,1952356,221013,1235516,483534,8820,1899081,14021,18504,...,921837,920415,493644,375242,178246,98660,104650,10495,4096,1100.0
6,1101.0,中央区,237627,23997,160339,50536,1891,235356,1722,1995,...,132006,131819,52695,73164,16633,9542,14403,1723,572,1101.0
7,1102.0,北区,285321,32916,178950,68960,2023,275495,1848,2558,...,133662,133477,70121,55247,25763,14221,14688,1316,571,1102.0
8,1103.0,東区,261912,30716,168302,61448,1121,255507,2204,2449,...,124425,124251,65606,51879,22696,12156,14050,1608,608,1103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,47361.0,久米島町,7755,1218,4475,2052,34,...,58,109,...,3365,3360,1798,1225,775,331,425,48,18,47361.0
1964,47362.0,八重瀬町,29066,5315,18012,5610,60,8212,405,230,...,9625,9598,6538,1915,2000,744,708,177,58,47362.0
1965,47375.0,多良間村,1194,208,671,315,14,...,12,15,...,470,470,268,142,118,49,51,3,1,47375.0
1966,47381.0,竹富町,3998,680,2469,798,38,...,57,48,...,2122,2121,832,1156,281,127,210,39,12,47381.0


In [3]:
population["市区町村名"].values

array(['北海道', '札幌市', '  中央区', ..., '多良間村', '竹富町', '与那国町'], dtype=object)

In [4]:
last = ""
prefecture = []
for place in population["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_population = population.copy()
custom_population["都道府県"] = prefecture
custom_population = custom_population.drop(custom_population[custom_population["市区町村名"] == custom_population["都道府県"]].index, axis=0)

custom_population = custom_population.drop(['婚姻件数', '離婚件数', '市区-町村-ｺｰﾄﾞ',
                                            '65歳以上の世帯員の-いる核家族世帯数', '核家族-世帯数',
                                            '高齢夫婦-世帯数', '高齢単身-世帯数', '人口集中-地区人口',
                                            "世帯数", "一般世帯数", "単独世帯数", "市 区 町 村"], axis=1)

custom_population = custom_population.set_index("都道府県")
custom_population = custom_population.reset_index()

custom_population = custom_population.astype({
                                            "都道府県": str,
                                            "市区町村名": str,
                                            "人口総数": int,
                                            "15歳未満-人口": int,
                                            "15～64歳-人口": int,
                                            "65歳以上-人口": int,
                                            "外国人人口": int,
                                            "出生数": int,
                                            "死亡数": int,
                                            "転入者数": int,
                                            "転出者数": int,
                                            "昼間人口": int,
                                            })
custom_population

,都道府県,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,出生数,死亡数,転入者数,転出者数,昼間人口
0,北海道,札幌市,1952356,221013,1235516,483534,8820,14021,18504,118336,109199,1959740
1,北海道,中央区,237627,23997,160339,50536,1891,1722,1995,20274,18490,386602
2,北海道,北区,285321,32916,178950,68960,2023,1848,2558,15580,13752,273734
3,北海道,東区,261912,30716,168302,61448,1121,2204,2449,15466,13889,247799
4,北海道,白石区,209584,23590,136879,47879,595,1849,2027,14188,13184,206604
...,...,...,...,...,...,...,...,...,...,...,...,...
1912,沖縄県,久米島町,7755,1218,4475,2052,34,58,109,428,516,7790
1913,沖縄県,八重瀬町,29066,5315,18012,5610,60,405,230,1563,1348,24992
1914,沖縄県,多良間村,1194,208,671,315,14,12,15,59,65,1180
1915,沖縄県,竹富町,3998,680,2469,798,38,57,48,517,502,4197


In [5]:
custom_population.to_excel("./custom-data/population/custom_population.xlsx", index=False)

In [6]:
wifi = pd.read_csv("./custom-data/wifi/470007_public_wireless_lan.csv")
wifi

,都道府県コード又は市区町村コード,NO,都道府県名,市区町村名,名称,名称_カナ,名称_英語,住所,方書,緯度,経度,設置者,電話番号,内線番号,SSID,提供エリア,URL,備考
0,472018,NaN,沖縄県,那覇市,今酒屋米丸,NaN,NaN,那覇市 港町2-3-8,NaN,26.243268,127.680657,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
1,472018,NaN,沖縄県,那覇市,沖縄鮮魚卸琉通協同組合,NaN,NaN,那覇市 港町1-1-18,NaN,26.251666,127.720086,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2,472018,NaN,沖縄県,那覇市,居酒屋 イカリ亭,NaN,NaN,那覇市 曙1-20-25,NaN,26.238608,127.683046,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
3,472018,NaN,沖縄県,那覇市,M’sニューポート,NaN,NaN,那覇市 曙2-1-1,NaN,26.240959,127.689195,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
4,472018,NaN,沖縄県,那覇市,バリバリ,NaN,NaN,那覇市 曙1-13-17,1F,26.236705,127.681789,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,472077,NaN,沖縄県,石垣市,石垣SSグループ 白保サービスステーション,NaN,NaN,石垣市 白保764,NaN,24.338209,124.160188,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2873,472077,NaN,沖縄県,石垣市,新石垣空港 国際線,NaN,NaN,石垣市 白保,NaN,24.396430,124.244996,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2874,472077,NaN,沖縄県,石垣市,石垣SSグループ 磯辺サービスステーション,NaN,NaN,石垣市 宮良1054-44,NaN,24.358280,124.208498,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN
2875,472077,NaN,沖縄県,石垣市,石垣SSグループ 川平サービスステーション,NaN,NaN,石垣市 川平853-1,NaN,24.463770,124.141494,沖縄県,NaN,NaN,Be.Okinawa_Free_Wi-Fi,NaN,NaN,NaN


In [7]:
custom_wifi = wifi.copy()[["都道府県名", "市区町村名", "名称", "住所"]]

custom_wifi["市区町村名"] = custom_wifi["市区町村名"].str.strip().str.replace(" ", "").str.replace("　", "")
custom_wifi

,都道府県名,市区町村名,名称,住所
0,沖縄県,那覇市,今酒屋米丸,那覇市 港町2-3-8
1,沖縄県,那覇市,沖縄鮮魚卸琉通協同組合,那覇市 港町1-1-18
2,沖縄県,那覇市,居酒屋 イカリ亭,那覇市 曙1-20-25
3,沖縄県,那覇市,M’sニューポート,那覇市 曙2-1-1
4,沖縄県,那覇市,バリバリ,那覇市 曙1-13-17
...,...,...,...,...
2872,沖縄県,石垣市,石垣SSグループ 白保サービスステーション,石垣市 白保764
2873,沖縄県,石垣市,新石垣空港 国際線,石垣市 白保
2874,沖縄県,石垣市,石垣SSグループ 磯辺サービスステーション,石垣市 宮良1054-44
2875,沖縄県,石垣市,石垣SSグループ 川平サービスステーション,石垣市 川平853-1


In [8]:
custom_wifi.to_excel("./custom-data/wifi/custom_470007_public_wireless_lan.xlsx", index=False)

In [9]:
economy = pd.read_excel("./custom-data/economy/2018-c.xls", header=2)
economy = economy.dropna(how="any", axis=0)

economy.columns = economy.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
economy["市区町村名"] = economy["市区町村名"].apply(lambda x: x.replace("\u3000", ""))

last = ""
prefecture = []
for place in economy["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_economy = economy.copy()
custom_economy["都道府県"] = prefecture
custom_economy = custom_economy.drop(custom_economy[custom_economy["市区町村名"] == custom_economy["都道府県"]].index, axis=0)

custom_economy = custom_economy.set_index("都道府県")
custom_economy = custom_economy.reset_index()


custom_economy = custom_economy[[
        '都道府県', '市区町村名', '事業所数',
        '第２次産業事業所数', '第３次産業事業所数',
       ]]

custom_economy["市区町村名"] = custom_economy["市区町村名"].str.strip()

custom_economy

,都道府県,市区町村名,事業所数,第２次産業事業所数,第３次産業事業所数
0,北海道,札幌市,76604,9321,67178
1,北海道,中央区,23138,1132,21971
2,北海道,北区,9331,1440,7883
3,北海道,東区,8964,1726,7232
4,北海道,白石区,8276,1432,6837
...,...,...,...,...,...
1912,沖縄県,久米島町,604,147,440
1913,沖縄県,八重瀬町,790,133,641
1914,沖縄県,多良間村,87,14,71
1915,沖縄県,竹富町,489,37,441


In [10]:
custom_economy.to_excel("./custom-data/economy/custom_2018-c.xlsx", index=False)

In [11]:
store = pd.read_excel("./custom-data/store/2018-h.xls", header=2)

store = store.dropna(how="any", axis=0)

store.columns = store.columns.str.replace("Unnamed: 1", "市区町村名").str.replace("\n", "-")
store["市区町村名"] = store["市区町村名"].apply(lambda x: x.replace("\u3000", ""))

last = ""
prefecture = []
for place in store["市区町村名"].values:
    if place[-1] in ["都", "道", "府", "県"]:
        last = place
        prefecture.append(last)
    else:
        prefecture.append(last)

custom_store = store.copy()
custom_store["都道府県"] = prefecture
custom_store = custom_store.drop(custom_store[custom_store["市区町村名"] == custom_store["都道府県"]].index, axis=0)

custom_store = custom_store.set_index("都道府県")
custom_store = custom_store.reset_index()

custom_store = custom_store[[
        '都道府県', '市区町村名',
        '小売店数', '飲食店数',
        ]]

custom_store["市区町村名"] = custom_store["市区町村名"].str.strip()

custom_store

,都道府県,市区町村名,小売店数,飲食店数
0,北海道,札幌市,11498,9124
1,北海道,中央区,3262,4327
2,北海道,北区,1436,972
3,北海道,東区,1359,631
4,北海道,白石区,1095,620
...,...,...,...,...
1912,沖縄県,久米島町,111,58
1913,沖縄県,八重瀬町,155,53
1914,沖縄県,多良間村,13,5
1915,沖縄県,竹富町,58,57


In [12]:
custom_store.to_excel("./custom-data/store/custom_2018-h.xlsx", index=False)

In [13]:
osaka_wifi = pd.read_csv("./custom-data/wifi/270008_public_wireless_lan_20230228.csv")


custom_osaka_wifi = osaka_wifi.copy()

import re
def get_city(_place: str):
    _place = _place.strip()

    found = re.match(r"^大阪府.*?[市町村]", _place).group(0)
    found = found.replace("大阪府", "").replace(" ", "").replace("　", "")

    if "郡" in found:
        found = found.split("郡")[1]
    return found

custom_osaka_wifi["市区町村名"] = custom_osaka_wifi["住所"].apply(get_city)
custom_osaka_wifi = custom_osaka_wifi[["都道府県名", "市区町村名", "名称", "住所"]]
custom_osaka_wifi

,都道府県名,市区町村名,名称,住所
0,大阪府,大阪市,ｓｈａｒｅ,大阪府大阪市北区堂山町 １６－４ パールレジャービルＢ１号
1,大阪府,大阪市,タバックケータリングサービス,大阪府大阪市東淀川区豊里 ７丁目３２－１１ 第二共和荘号１０１号
2,大阪府,堺市,ｈａｉｒｐｌａｃｅａｐｅｇｏ,大阪府堺市東区北野田 １０８４ ベルヒル北野田施設棟１２９号
3,大阪府,大阪市,道頓堀どむす相川店,大阪府大阪市東淀川区相川 ２丁目６－８
4,大阪府,大阪市,焼肉 炙,大阪府大阪市東淀川区小松 １丁目１０－２２ エクセレント上新１Ｆ
...,...,...,...,...
3455,大阪府,大阪市,南海電気鉄道 特急サザン 車両Wi-Fi,大阪府大阪市中央区難波5丁目1番60号
3456,大阪府,大阪市,南海電気鉄道 特急サザンプレミアム 車両Wi-Fi,大阪府大阪市中央区難波5丁目1番60号
3457,大阪府,大阪市,南海電気鉄道 特急りんかん 車両Wi-Fi,大阪府大阪市中央区難波5丁目1番60号
3458,大阪府,大阪市,南海電気鉄道 特急こうや 車両Wi-Fi,大阪府大阪市中央区難波5丁目1番60号


In [14]:
custom_osaka_wifi.groupby("市区町村名").count()

,都道府県名,名称,住所
市区町村名,,,
交野市,7,7,7
八尾市,79,79,79
千早赤阪村,5,5,5
吹田市,75,75,75
和泉市,41,41,41
四條畷市,12,12,12
堺市,262,262,262
大東市,33,33,33
大阪市,1822,1822,1822


In [15]:
custom_osaka_wifi.to_excel("./custom-data/wifi/custom_270008_public_wireless_lan_20230228.xlsx", index=False)

In [16]:
population.merge(custom_osaka_wifi, left_on="市区町村名", right_on="市区町村名", how="outer")

,市 区 町 村,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,人口集中-地区人口,出生数,死亡数,...,単独世帯数,65歳以上の世帯員の-いる核家族世帯数,高齢夫婦-世帯数,高齢単身-世帯数,婚姻件数,離婚件数,市区-町村-ｺｰﾄﾞ,都道府県名,名称,住所
0,1.0,北海道,5381733,608296,3190804,1558387,21676,4047281,35125,61906,...,909106,558618,327509,319408,24636,10476,1.0,NaN,NaN,NaN
1,1100.0,札幌市,1952356,221013,1235516,483534,8820,1899081,14021,18504,...,375242,178246,98660,104650,10495,4096,1100.0,NaN,NaN,NaN
2,1101.0,中央区,237627,23997,160339,50536,1891,235356,1722,1995,...,73164,16633,9542,14403,1723,572,1101.0,NaN,NaN,NaN
3,11105.0,中央区,98762,13246,65083,20026,1149,98762,905,721,...,14972,7954,3842,3543,579,141,11105.0,NaN,NaN,NaN
4,12101.0,中央区,205070,23938,134953,45880,3098,198595,1663,1821,...,39426,16817,8165,9039,1344,373,12101.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5377,47361.0,久米島町,7755,1218,4475,2052,34,...,58,109,...,1225,775,331,425,48,18,47361.0,NaN,NaN,NaN
5378,47362.0,八重瀬町,29066,5315,18012,5610,60,8212,405,230,...,1915,2000,744,708,177,58,47362.0,NaN,NaN,NaN
5379,47375.0,多良間村,1194,208,671,315,14,...,12,15,...,142,118,49,51,3,1,47375.0,NaN,NaN,NaN
5380,47381.0,竹富町,3998,680,2469,798,38,...,57,48,...,1156,281,127,210,39,12,47381.0,NaN,NaN,NaN


In [17]:
[a for a in population["市区町村名"] if "熊取町" in a]

['熊取町']

In [18]:
import pandas as pd

okinawa_wifi = pd.read_excel("./custom-data/wifi/custom_470007_public_wireless_lan.xlsx")
osaka_wifi = pd.read_excel("./custom-data/wifi/custom_270008_public_wireless_lan_20230228.xlsx")
population = pd.read_excel("./custom-data/population/custom_population.xlsx")
economy = pd.read_excel("./custom-data/economy/custom_2018-c.xlsx")
store = pd.read_excel("./custom-data/store/custom_2018-h.xlsx")


wifi = pd.concat([okinawa_wifi, osaka_wifi])

df = population.merge(economy, left_on="市区町村名", right_on="市区町村名", how="inner")
df = df.merge(store, left_on="市区町村名", right_on="市区町村名", how="inner")
df = df.drop([col for col in df.columns if "都道府県_" in col], axis=1)
df = df.drop(["都道府県"], axis=1)
df

,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,出生数,死亡数,転入者数,転出者数,昼間人口,事業所数,第２次産業事業所数,第３次産業事業所数,小売店数,飲食店数
0,札幌市,1952356,221013,1235516,483534,8820,14021,18504,118336,109199,1959740,76604,9321,67178,11498,9124
1,函館市,265979,27131,152154,85931,771,1532,3637,8718,9791,273408,13795,1839,11912,2689,1872
2,小樽市,121924,11171,65317,45240,411,512,1856,3121,3970,124293,6090,884,5197,1373,821
3,旭川市,339605,37173,191423,106444,613,2280,4060,10282,11029,341732,15201,2219,12906,2639,1840
4,室蘭市,88564,9185,49005,30118,297,503,1220,2999,3771,96865,4423,683,3739,764,669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7870,久米島町,7755,1218,4475,2052,34,58,109,428,516,7790,604,147,440,111,58
7871,八重瀬町,29066,5315,18012,5610,60,405,230,1563,1348,24992,790,133,641,155,53
7872,多良間村,1194,208,671,315,14,12,15,59,65,1180,87,14,71,13,5
7873,竹富町,3998,680,2469,798,38,57,48,517,502,4197,489,37,441,58,57


In [19]:
df["15歳未満-割合"] = df["15歳未満-人口"] / df["人口総数"]
df["15～64歳-割合"] = df["15～64歳-人口"] / df["人口総数"]
df["65歳以上-割合"] = df["65歳以上-人口"] / df["人口総数"]
df.sort_values("15歳未満-割合", ascending=False)

,市区町村名,人口総数,15歳未満-人口,15～64歳-人口,65歳以上-人口,外国人人口,出生数,死亡数,転入者数,転出者数,昼間人口,事業所数,第２次産業事業所数,第３次産業事業所数,小売店数,飲食店数,15歳未満-割合,15～64歳-割合,65歳以上-割合
535,朝日町,10560,2225,6017,2015,85,121,71,450,353,9355,732,165,556,46,22,0.210701,0.569792,0.190814
531,朝日町,10560,2225,6017,2015,85,121,71,450,353,9355,361,104,253,149,52,0.210701,0.569792,0.190814
536,朝日町,10560,2225,6017,2015,85,121,71,450,353,9355,306,83,221,81,25,0.210701,0.569792,0.190814
538,朝日町,10560,2225,6017,2015,85,121,71,450,353,9355,306,83,221,46,22,0.210701,0.569792,0.190814
537,朝日町,10560,2225,6017,2015,85,121,71,450,353,9355,306,83,221,149,52,0.210701,0.569792,0.190814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,飯舘村,41,0,0,41,0,54,86,39,133,3246,1,0,1,0,0,0.000000,0.000000,1.000000
633,富岡町,0,0,0,0,0,103,138,43,274,2961,5,3,2,0,0,NaN,NaN,NaN
635,大熊町,0,0,0,0,0,101,88,53,162,3701,...,...,...,...,...,NaN,NaN,NaN
636,双葉町,0,0,0,0,0,44,70,40,83,322,0,0,0,0,0,NaN,NaN,NaN


In [20]:
wifi.to_excel("./custom-data/clean/wifi.xlsx", index=False)
df.to_excel("./custom-data/clean/osaka_okinawa_info.xlsx", index=False)